# Notebook for data analysis and Cleaning

In [1]:
import os
from langchain_groq import ChatGroq
from dotenv import load_dotenv
# from langchain_mistralai import MistralAIEmbeddings
import pandas as pd
import json
from langchain.embeddings import HuggingFaceEmbeddings
import numpy as np
import sys

load_dotenv()
model_id = "llama-3.1-8b-instant" # 	qwen/qwen3-32b  openai/gpt-oss-120b llama3-8b-8192
groq_api_key = os.getenv("GROQ_API_KEY")

llm = ChatGroq(model=model_id,
            temperature=0,
            max_tokens=None,
            timeout=None,
            max_retries=2,
            verbose=1)

# Create embedding model
embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")


sys.path.append('../src')

/var/folders/0z/6z_9lrtd6j3_gnlsyv9rb5b00000gn/T/ipykernel_87092/2473950239.py:23: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")


## Initial data processing

In [2]:
import pandas as pd
import json

# Read the JSON file
with open('../data/convfinqa_dataset.json', 'r') as f:
    data = json.load(f)

# Convert to DataFrame
df_train = pd.DataFrame(data['train'])
def_dev = pd.DataFrame(data['dev'])
df = pd.concat([df_train, def_dev], ignore_index=True)

# Display the first few rows
print('Shape of the dataframe: ', df.shape)
df.head()

Shape of the dataframe:  (3458, 4)


,id,doc,dialogue,features
0,Single_JKHY/2009/page_28.pdf-3,{'pre_text': '26 | 2009 annual report in fisca...,{'conv_questions': ['what is the net cash from...,"{'num_dialogue_turns': 4, 'has_type2_question'..."
1,Single_RSG/2008/page_114.pdf-2,{'pre_text': 'substantially all of the goodwil...,{'conv_questions': ['what were revenues in 200...,"{'num_dialogue_turns': 4, 'has_type2_question'..."
2,Single_AAPL/2002/page_23.pdf-1,{'pre_text': 'in a new business model such as ...,{'conv_questions': ['what was the total of net...,"{'num_dialogue_turns': 4, 'has_type2_question'..."
3,Single_UPS/2009/page_33.pdf-2,{'pre_text': '( 1 ) includes shares repurchase...,{'conv_questions': ['what was the change in th...,"{'num_dialogue_turns': 6, 'has_type2_question'..."
4,Double_UPS/2009/page_33.pdf,{'pre_text': '( 1 ) includes shares repurchase...,{'conv_questions': ['what was the fluctuation ...,"{'num_dialogue_turns': 7, 'has_type2_question'..."


In [3]:
df.iloc[2]['doc']

{'pre_text': 'in a new business model such as the retail segment is inherently risky , particularly in light of the significant investment involved , the current economic climate , and the fixed nature of a substantial portion of the retail segment\'s operating expenses . results for this segment are dependent upon a number of risks and uncertainties , some of which are discussed below under the heading "factors that may affect future results and financial condition." backlog in the company\'s experience , the actual amount of product backlog at any particular time is not a meaningful indication of its future business prospects . in particular , backlog often increases in anticipation of or immediately following new product introductions because of over- ordering by dealers anticipating shortages . backlog often is reduced once dealers and customers believe they can obtain sufficient supply . because of the foregoing , backlog cannot be considered a reliable indicator of the company\'s

## Summarize each document 

In [ ]:
from joblib import Parallel, delayed
import math
import time

llm = ChatGroq(model='llama-3.1-8b-instant', temperature=0, max_tokens=60, timeout=None, max_retries=2, verbose=1)

def summarize_doc(doc,llm):

    response = llm.invoke(f"""
            System: You are a helpful assistant that gives a topic to a document and describes it in a one or two sentences.\n
            
            User: Please write a topic for the following document, together with a few sentences description on what is it about:\n{doc}.

            Please keep the description concise and to the point (up to 50 words).

            Please only return the following:
            
            Topic: 

            Description:
            """)
    return str(response.content)

batch_size = 20  # You can adjust this value
num_batches = math.ceil(len(df) / batch_size)

summaries = []
for i in range(num_batches):
    batch = df.iloc[i*batch_size : (i+1)*batch_size]
    batch_summaries = Parallel(n_jobs=-1, backend="threading")(delayed(summarize_doc)(doc, llm) for doc in batch['doc'])
    summaries.extend(batch_summaries)
    print(f'Batch {i+1} of {num_batches} done.')
    time.sleep(5)  # To avoid hitting rate limits


df['summary'] = summaries
# for parallel processing need to use another technology like joblib 

Batch 1 of 173 done.
Batch 2 of 173 done.
Batch 3 of 173 done.
Batch 4 of 173 done.
Batch 5 of 173 done.
Batch 6 of 173 done.
Batch 7 of 173 done.
Batch 8 of 173 done.
Batch 9 of 173 done.
Batch 10 of 173 done.
Batch 11 of 173 done.
Batch 12 of 173 done.
Batch 13 of 173 done.
Batch 14 of 173 done.
Batch 15 of 173 done.
Batch 16 of 173 done.
Batch 17 of 173 done.
Batch 18 of 173 done.
Batch 19 of 173 done.
Batch 20 of 173 done.
Batch 21 of 173 done.
Batch 22 of 173 done.
Batch 23 of 173 done.
Batch 24 of 173 done.
Batch 25 of 173 done.
Batch 26 of 173 done.
Batch 27 of 173 done.
Batch 28 of 173 done.
Batch 29 of 173 done.
Batch 30 of 173 done.
Batch 31 of 173 done.
Batch 32 of 173 done.
Batch 33 of 173 done.
Batch 34 of 173 done.
Batch 35 of 173 done.
Batch 36 of 173 done.
Batch 37 of 173 done.
Batch 38 of 173 done.
Batch 39 of 173 done.
Batch 40 of 173 done.
Batch 41 of 173 done.
Batch 42 of 173 done.
Batch 43 of 173 done.
Batch 44 of 173 done.
Batch 45 of 173 done.
Batch 46 of 173 don

In [42]:
df.to_csv('../data/convfinqa_wsummary.csv', index=False)

## Add embeddings

In [3]:
# Load the DataFrame from the CSV file
df_s = pd.read_csv('../data/convfinqa_wsummary.csv')
df_s.head()

,id,doc,dialogue,features,summary
0,Single_JKHY/2009/page_28.pdf-3,{'pre_text': '26 | 2009 annual report in fisca...,{'conv_questions': ['what is the net cash from...,"{'num_dialogue_turns': 4, 'has_type2_question'...",Topic: \nFinancial Performance of a Credit Uni...
1,Single_RSG/2008/page_114.pdf-2,{'pre_text': 'substantially all of the goodwil...,{'conv_questions': ['what were revenues in 200...,"{'num_dialogue_turns': 4, 'has_type2_question'...",Topic: \nPro Forma Financial Information and A...
2,Single_AAPL/2002/page_23.pdf-1,{'pre_text': 'in a new business model such as ...,{'conv_questions': ['what was the total of net...,"{'num_dialogue_turns': 4, 'has_type2_question'...",Topic: \nAnalysis of Gross Margin and Operatin...
3,Single_UPS/2009/page_33.pdf-2,{'pre_text': '( 1 ) includes shares repurchase...,{'conv_questions': ['what was the change in th...,"{'num_dialogue_turns': 6, 'has_type2_question'...",Topic: Shareowner Return Performance Graph for...
4,Double_UPS/2009/page_33.pdf,{'pre_text': '( 1 ) includes shares repurchase...,{'conv_questions': ['what was the fluctuation ...,"{'num_dialogue_turns': 7, 'has_type2_question'...",Topic: Shareowner Return Performance Graph for...


In [21]:
# create embedding for the summaries
df_s["summary_emb"] = df_s["summary"].apply(lambda x: embedding_model.embed_query(x))



In [24]:
df_s.to_csv('../data/convfinqa_wembed.csv', index=False)
df_s.head()

,id,doc,dialogue,features,summary,summary_emb
0,Single_JKHY/2009/page_28.pdf-3,{'pre_text': '26 | 2009 annual report in fisca...,{'conv_questions': ['what is the net cash from...,"{'num_dialogue_turns': 4, 'has_type2_question'...",Topic: \nFinancial Performance of a Credit Uni...,"[0.013813999481499195, -0.07805461436510086, 0..."
1,Single_RSG/2008/page_114.pdf-2,{'pre_text': 'substantially all of the goodwil...,{'conv_questions': ['what were revenues in 200...,"{'num_dialogue_turns': 4, 'has_type2_question'...",Topic: \nPro Forma Financial Information and A...,"[-0.012922744266688824, -0.06456907838582993, ..."
2,Single_AAPL/2002/page_23.pdf-1,{'pre_text': 'in a new business model such as ...,{'conv_questions': ['what was the total of net...,"{'num_dialogue_turns': 4, 'has_type2_question'...",Topic: \nAnalysis of Gross Margin and Operatin...,"[0.0043345107696950436, -0.034141842275857925,..."
3,Single_UPS/2009/page_33.pdf-2,{'pre_text': '( 1 ) includes shares repurchase...,{'conv_questions': ['what was the change in th...,"{'num_dialogue_turns': 6, 'has_type2_question'...",Topic: Shareowner Return Performance Graph for...,"[-0.002877976046875119, -0.04268963262438774, ..."
4,Double_UPS/2009/page_33.pdf,{'pre_text': '( 1 ) includes shares repurchase...,{'conv_questions': ['what was the fluctuation ...,"{'num_dialogue_turns': 7, 'has_type2_question'...",Topic: Shareowner Return Performance Graph for...,"[-0.002877976046875119, -0.04268963262438774, ..."


In [ ]:
# simple search through the data
row = df_s[df_s['features'].apply(lambda x: eval(str(x))['has_type2_question'])].iloc[30]

print('summary: ', row['summary'])
print('Features: ', row['features'])
print('Dialogue: ', row['dialogue'])

summary:  Topic: 
Financial Statement Analysis: Accruals, Litigation Liabilities, and Asset Impairment

Description: This document discusses the financial statement impact of accruals for sales returns, rebates, and discounts, product litigation liabilities, and asset impairment, including the factors considered in developing
Features:  {'num_dialogue_turns': 4, 'has_type2_question': True, 'has_duplicate_columns': False, 'has_non_numeric_values': False}
Dialogue:  {'conv_questions': ['what was the change in the total of u.s . pharmaceutical sales return, rebate, and discount liability balances, including managed care, medicare, and medicaid from 2017 to 2018?', 'and what is this change as a percentage of that total in 2017?', 'in that same period, what was the variation in the reduction of net sales due to sales returns discounts and rebates?', 'and what percentage did this variation represent in relation to that reduction in 2017?'], 'conv_answers': ['506.2', '12%', '1926.2', '18%'], 

## Search relevant summaries as a function

In [62]:
# Load the DataFrame from the CSV file
df_e = pd.read_csv('../data/convfinqa_wembed.csv')
df_e.head()

,id,doc,dialogue,features,summary,summary_emb
0,Single_JKHY/2009/page_28.pdf-3,{'pre_text': '26 | 2009 annual report in fisca...,{'conv_questions': ['what is the net cash from...,"{'num_dialogue_turns': 4, 'has_type2_question'...",Topic: \nFinancial Performance of a Credit Uni...,[ 1.38139995e-02 -7.80546144e-02 3.70453261e-...
1,Single_RSG/2008/page_114.pdf-2,{'pre_text': 'substantially all of the goodwil...,{'conv_questions': ['what were revenues in 200...,"{'num_dialogue_turns': 4, 'has_type2_question'...",Topic: \nPro Forma Financial Information and A...,[-1.29227443e-02 -6.45690784e-02 1.02868928e-...
2,Single_AAPL/2002/page_23.pdf-1,{'pre_text': 'in a new business model such as ...,{'conv_questions': ['what was the total of net...,"{'num_dialogue_turns': 4, 'has_type2_question'...",Topic: \nAnalysis of Gross Margin and Operatin...,[ 4.33451077e-03 -3.41418423e-02 6.81793466e-...
3,Single_UPS/2009/page_33.pdf-2,{'pre_text': '( 1 ) includes shares repurchase...,{'conv_questions': ['what was the change in th...,"{'num_dialogue_turns': 6, 'has_type2_question'...",Topic: Shareowner Return Performance Graph for...,[-2.87797605e-03 -4.26896326e-02 -2.22300575e-...
4,Double_UPS/2009/page_33.pdf,{'pre_text': '( 1 ) includes shares repurchase...,{'conv_questions': ['what was the fluctuation ...,"{'num_dialogue_turns': 7, 'has_type2_question'...",Topic: Shareowner Return Performance Graph for...,[-2.87797605e-03 -4.26896326e-02 -2.22300575e-...


In [5]:
print(type(df_e["summary_emb"].iloc[0]))
print(df_e["summary_emb"].iloc[0][:5]) 
print('Need to parse the embeddings from string to list of floats')

<class 'str'>
[ 1.3
Need to parse the embeddings from string to list of floats


In [6]:
import re
import numpy as np

def parse_embedding(s):
    if isinstance(s, np.ndarray):
        return s
    if isinstance(s, list):
        return np.array(s, dtype=float)
    if not isinstance(s, str):
        raise ValueError(f"Unexpected embedding type: {type(s)}")

    # Remove brackets and extra spaces
    s = s.strip().replace("[", "").replace("]", "")
    # Split on spaces (one or more)
    nums = re.split(r"\s+", s.strip())
    # Convert to float
    return np.array([float(x) for x in nums if x != ""], dtype=float)

df_e["summary_emb"] = df_e["summary_emb"].apply(parse_embedding)


In [ ]:
from numpy.linalg import norm

def find_relevant_document_fast(embedding_model, query, df, embed_col, top_k=3):
    query_emb = np.array(embedding_model.embed_query(query), dtype=float)
    df_e[embed_col] = df_e[embed_col].apply(parse_embedding)

    sims = df[embed_col].apply(lambda x: np.dot(x, query_emb) / (norm(x) * norm(query_emb)))
    top_docs = df.iloc[np.argsort(sims)[-top_k:][::-1]]
    return top_docs

In [ ]:
query = "What was S&P 500's stock price trend in the last quarter in 2012?"
relevant_docs = find_relevant_document_fast(embedding_model, query, df_e, 'summary_emb', top_k=2)
for i, row in relevant_docs.iterrows():
    print(f"Document {i}:")
    print("Summary:", row['summary'])
    print("Dialogue:", row['dialogue'])
    print("Features:", row['features'])
    print("\n---\n")    

Document 842:
Summary: Topic: 
Stock Total Return Performance Comparison

Description: This document compares the total return performance of our company's stock with the S&P 500 and a peer group index over a five-year period from December 31, 2012, to December 31, 2017.
Dialogue: {'conv_questions': ['what was the stock total return performance for hum in 2014?', 'and what was it in 2013?', 'by what amount, then, did that performance increase over the year?', 'and in that same year of 2014, what was the highest stock total return performance between all stocks?', 'by how much did this performance change since 2012?'], 'conv_answers': ['214', '152', '62', '214', '114%'], 'turn_program': ['214', '152', 'subtract(214, 152)', '214', 'subtract(214, 100)'], 'executed_answers': [214.0, 152.0, 62.0, 214.0, 114.0], 'qa_split': [False, False, False, True, True]}
Features: {'num_dialogue_turns': 5, 'has_type2_question': True, 'has_duplicate_columns': False, 'has_non_numeric_values': False}

---



## Restructure the FinQA dataset

In [65]:
import ast

# Extract the dictionary from the string in df_e.iloc[0]['dialogue']
dialogue_str = df_e.iloc[0]['dialogue']
dialogue_dict = ast.literal_eval(dialogue_str)

print(type(dialogue_dict))  # Should be <class 'dict'>
print(dialogue_dict)

<class 'dict'>
{'conv_questions': ['what is the net cash from operating activities in 2009?', 'what about in 2008?', 'what is the difference?', 'what percentage change does this represent?'], 'conv_answers': ['206588', '181001', '25587', '14.1%'], 'turn_program': ['206588', '181001', 'subtract(206588, 181001)', 'subtract(206588, 181001), divide(#0, 181001)'], 'executed_answers': [206588.0, 181001.0, 25587.0, 0.14136], 'qa_split': [False, False, False, False]}


In [66]:
df_e['dialogue_dict'] = df_e['dialogue'].apply(lambda x: ast.literal_eval(x))
# For each key in the first dictionary, create a new column
for key in df_e.iloc[0]['dialogue_dict'].keys():
    df_e[key] = df_e['dialogue_dict'].apply(lambda d: d[key])

df_e.head()

,id,doc,dialogue,features,summary,summary_emb,dialogue_dict,conv_questions,conv_answers,turn_program,executed_answers,qa_split
0,Single_JKHY/2009/page_28.pdf-3,{'pre_text': '26 | 2009 annual report in fisca...,{'conv_questions': ['what is the net cash from...,"{'num_dialogue_turns': 4, 'has_type2_question'...",Topic: \nFinancial Performance of a Credit Uni...,"[0.0138139995, -0.0780546144, 0.0370453261, 0....",{'conv_questions': ['what is the net cash from...,[what is the net cash from operating activitie...,"[206588, 181001, 25587, 14.1%]","[206588, 181001, subtract(206588, 181001), sub...","[206588.0, 181001.0, 25587.0, 0.14136]","[False, False, False, False]"
1,Single_RSG/2008/page_114.pdf-2,{'pre_text': 'substantially all of the goodwil...,{'conv_questions': ['what were revenues in 200...,"{'num_dialogue_turns': 4, 'has_type2_question'...",Topic: \nPro Forma Financial Information and A...,"[-0.0129227443, -0.0645690784, 0.0102868928, -...",{'conv_questions': ['what were revenues in 200...,"[what were revenues in 2008?, what were they i...","[9362.2, 9244.9, 117.3, 1.3%]","[9362.2, 9244.9, subtract(9362.2, 9244.9), sub...","[9362.2, 9244.9, 117.3, 0.01269]","[False, False, False, False]"
2,Single_AAPL/2002/page_23.pdf-1,{'pre_text': 'in a new business model such as ...,{'conv_questions': ['what was the total of net...,"{'num_dialogue_turns': 4, 'has_type2_question'...",Topic: \nAnalysis of Gross Margin and Operatin...,"[0.00433451077, -0.0341418423, 0.0681793466, -...",{'conv_questions': ['what was the total of net...,"[what was the total of net sales in 2001?, and...","[5363, 7983, -2620, -32%]","[5363, 7983, subtract(5363, 7983), subtract(53...","[5363.0, 7983.0, -2620.0, -0.3282]","[False, False, False, False]"
3,Single_UPS/2009/page_33.pdf-2,{'pre_text': '( 1 ) includes shares repurchase...,{'conv_questions': ['what was the change in th...,"{'num_dialogue_turns': 6, 'has_type2_question'...",Topic: Shareowner Return Performance Graph for...,"[-0.00287797605, -0.0426896326, -0.00222300575...",{'conv_questions': ['what was the change in th...,[what was the change in the performance of the...,"[-24.05, -24.05%, 102.11, 2.11, 2.11%, -26.16%]","[subtract(75.95, const_100), subtract(75.95, c...","[-24.05, -0.2405, 102.11, 2.11, 0.0211, -0.2616]","[False, False, False, False, False, False]"
4,Double_UPS/2009/page_33.pdf,{'pre_text': '( 1 ) includes shares repurchase...,{'conv_questions': ['what was the fluctuation ...,"{'num_dialogue_turns': 7, 'has_type2_question'...",Topic: Shareowner Return Performance Graph for...,"[-0.00287797605, -0.0426896326, -0.00222300575...",{'conv_questions': ['what was the fluctuation ...,[what was the fluctuation of the performance p...,"[-8.94, -8.9%, -24.05, -24.05%, 2.11, 2.11%, -...","[subtract(91.06, const_100), subtract(91.06, c...","[-8.94, -0.0894, -24.05, -0.2405, 2.11, 0.0211...","[False, False, True, True, True, True, True]"


In [67]:
def dict_to_doctext(dialogue_dict):
    lines = []
    for i in range(len(dialogue_dict['conv_questions'])):
        q = dialogue_dict['conv_questions'][i]
        reason = dialogue_dict['turn_program'][i]
        a = dialogue_dict['executed_answers'][i]
        lines.append(f"Q: {q}\nReason: {reason}\nA: {a}")
    return '"""\n' + '\n'.join(lines) + '\n"""'

# Example usage:
doctext = dict_to_doctext(df_e['dialogue_dict'].iloc[0])
print(doctext)

"""
Q: what is the net cash from operating activities in 2009?
Reason: 206588
A: 206588.0
Q: what about in 2008?
Reason: 181001
A: 181001.0
Q: what is the difference?
Reason: subtract(206588, 181001)
A: 25587.0
Q: what percentage change does this represent?
Reason: subtract(206588, 181001), divide(#0, 181001)
A: 0.14136
"""


In [68]:
df_e['conv_text_full'] = df_e['dialogue_dict'].apply(lambda x: dict_to_doctext(x))
df_e.head()

,id,doc,dialogue,features,summary,summary_emb,dialogue_dict,conv_questions,conv_answers,turn_program,executed_answers,qa_split,conv_text_full
0,Single_JKHY/2009/page_28.pdf-3,{'pre_text': '26 | 2009 annual report in fisca...,{'conv_questions': ['what is the net cash from...,"{'num_dialogue_turns': 4, 'has_type2_question'...",Topic: \nFinancial Performance of a Credit Uni...,"[0.0138139995, -0.0780546144, 0.0370453261, 0....",{'conv_questions': ['what is the net cash from...,[what is the net cash from operating activitie...,"[206588, 181001, 25587, 14.1%]","[206588, 181001, subtract(206588, 181001), sub...","[206588.0, 181001.0, 25587.0, 0.14136]","[False, False, False, False]","""""""\nQ: what is the net cash from operating ac..."
1,Single_RSG/2008/page_114.pdf-2,{'pre_text': 'substantially all of the goodwil...,{'conv_questions': ['what were revenues in 200...,"{'num_dialogue_turns': 4, 'has_type2_question'...",Topic: \nPro Forma Financial Information and A...,"[-0.0129227443, -0.0645690784, 0.0102868928, -...",{'conv_questions': ['what were revenues in 200...,"[what were revenues in 2008?, what were they i...","[9362.2, 9244.9, 117.3, 1.3%]","[9362.2, 9244.9, subtract(9362.2, 9244.9), sub...","[9362.2, 9244.9, 117.3, 0.01269]","[False, False, False, False]","""""""\nQ: what were revenues in 2008?\nReason: 9..."
2,Single_AAPL/2002/page_23.pdf-1,{'pre_text': 'in a new business model such as ...,{'conv_questions': ['what was the total of net...,"{'num_dialogue_turns': 4, 'has_type2_question'...",Topic: \nAnalysis of Gross Margin and Operatin...,"[0.00433451077, -0.0341418423, 0.0681793466, -...",{'conv_questions': ['what was the total of net...,"[what was the total of net sales in 2001?, and...","[5363, 7983, -2620, -32%]","[5363, 7983, subtract(5363, 7983), subtract(53...","[5363.0, 7983.0, -2620.0, -0.3282]","[False, False, False, False]","""""""\nQ: what was the total of net sales in 200..."
3,Single_UPS/2009/page_33.pdf-2,{'pre_text': '( 1 ) includes shares repurchase...,{'conv_questions': ['what was the change in th...,"{'num_dialogue_turns': 6, 'has_type2_question'...",Topic: Shareowner Return Performance Graph for...,"[-0.00287797605, -0.0426896326, -0.00222300575...",{'conv_questions': ['what was the change in th...,[what was the change in the performance of the...,"[-24.05, -24.05%, 102.11, 2.11, 2.11%, -26.16%]","[subtract(75.95, const_100), subtract(75.95, c...","[-24.05, -0.2405, 102.11, 2.11, 0.0211, -0.2616]","[False, False, False, False, False, False]","""""""\nQ: what was the change in the performance..."
4,Double_UPS/2009/page_33.pdf,{'pre_text': '( 1 ) includes shares repurchase...,{'conv_questions': ['what was the fluctuation ...,"{'num_dialogue_turns': 7, 'has_type2_question'...",Topic: Shareowner Return Performance Graph for...,"[-0.00287797605, -0.0426896326, -0.00222300575...",{'conv_questions': ['what was the fluctuation ...,[what was the fluctuation of the performance p...,"[-8.94, -8.9%, -24.05, -24.05%, 2.11, 2.11%, -...","[subtract(91.06, const_100), subtract(91.06, c...","[-8.94, -0.0894, -24.05, -0.2405, 2.11, 0.0211...","[False, False, True, True, True, True, True]","""""""\nQ: what was the fluctuation of the perfor..."


In [69]:
df_e.iloc[3]['features']

"{'num_dialogue_turns': 6, 'has_type2_question': False, 'has_duplicate_columns': False, 'has_non_numeric_values': False}"

In [70]:
df_e['features_dict'] = df_e['features'].apply(lambda x: ast.literal_eval(x))
# For each key in the first dictionary, create a new column
for key in df_e.iloc[0]['features_dict'].keys():
    df_e[key] = df_e['features_dict'].apply(lambda d: d[key])

df_e.head()

,id,doc,dialogue,features,summary,summary_emb,dialogue_dict,conv_questions,conv_answers,turn_program,executed_answers,qa_split,conv_text_full,features_dict,num_dialogue_turns,has_type2_question,has_duplicate_columns,has_non_numeric_values
0,Single_JKHY/2009/page_28.pdf-3,{'pre_text': '26 | 2009 annual report in fisca...,{'conv_questions': ['what is the net cash from...,"{'num_dialogue_turns': 4, 'has_type2_question'...",Topic: \nFinancial Performance of a Credit Uni...,"[0.0138139995, -0.0780546144, 0.0370453261, 0....",{'conv_questions': ['what is the net cash from...,[what is the net cash from operating activitie...,"[206588, 181001, 25587, 14.1%]","[206588, 181001, subtract(206588, 181001), sub...","[206588.0, 181001.0, 25587.0, 0.14136]","[False, False, False, False]","""""""\nQ: what is the net cash from operating ac...","{'num_dialogue_turns': 4, 'has_type2_question'...",4,False,False,False
1,Single_RSG/2008/page_114.pdf-2,{'pre_text': 'substantially all of the goodwil...,{'conv_questions': ['what were revenues in 200...,"{'num_dialogue_turns': 4, 'has_type2_question'...",Topic: \nPro Forma Financial Information and A...,"[-0.0129227443, -0.0645690784, 0.0102868928, -...",{'conv_questions': ['what were revenues in 200...,"[what were revenues in 2008?, what were they i...","[9362.2, 9244.9, 117.3, 1.3%]","[9362.2, 9244.9, subtract(9362.2, 9244.9), sub...","[9362.2, 9244.9, 117.3, 0.01269]","[False, False, False, False]","""""""\nQ: what were revenues in 2008?\nReason: 9...","{'num_dialogue_turns': 4, 'has_type2_question'...",4,False,False,False
2,Single_AAPL/2002/page_23.pdf-1,{'pre_text': 'in a new business model such as ...,{'conv_questions': ['what was the total of net...,"{'num_dialogue_turns': 4, 'has_type2_question'...",Topic: \nAnalysis of Gross Margin and Operatin...,"[0.00433451077, -0.0341418423, 0.0681793466, -...",{'conv_questions': ['what was the total of net...,"[what was the total of net sales in 2001?, and...","[5363, 7983, -2620, -32%]","[5363, 7983, subtract(5363, 7983), subtract(53...","[5363.0, 7983.0, -2620.0, -0.3282]","[False, False, False, False]","""""""\nQ: what was the total of net sales in 200...","{'num_dialogue_turns': 4, 'has_type2_question'...",4,False,False,False
3,Single_UPS/2009/page_33.pdf-2,{'pre_text': '( 1 ) includes shares repurchase...,{'conv_questions': ['what was the change in th...,"{'num_dialogue_turns': 6, 'has_type2_question'...",Topic: Shareowner Return Performance Graph for...,"[-0.00287797605, -0.0426896326, -0.00222300575...",{'conv_questions': ['what was the change in th...,[what was the change in the performance of the...,"[-24.05, -24.05%, 102.11, 2.11, 2.11%, -26.16%]","[subtract(75.95, const_100), subtract(75.95, c...","[-24.05, -0.2405, 102.11, 2.11, 0.0211, -0.2616]","[False, False, False, False, False, False]","""""""\nQ: what was the change in the performance...","{'num_dialogue_turns': 6, 'has_type2_question'...",6,False,False,False
4,Double_UPS/2009/page_33.pdf,{'pre_text': '( 1 ) includes shares repurchase...,{'conv_questions': ['what was the fluctuation ...,"{'num_dialogue_turns': 7, 'has_type2_question'...",Topic: Shareowner Return Performance Graph for...,"[-0.00287797605, -0.0426896326, -0.00222300575...",{'conv_questions': ['what was the fluctuation ...,[what was the fluctuation of the performance p...,"[-8.94, -8.9%, -24.05, -24.05%, 2.11, 2.11%, -...","[subtract(91.06, const_100), subtract(91.06, c...","[-8.94, -0.0894, -24.05, -0.2405, 2.11, 0.0211...","[False, False, True, True, True, True, True]","""""""\nQ: what was the fluctuation of the perfor...","{'num_dialogue_turns': 7, 'has_type2_question'...",7,True,False,False


In [71]:
df_e.to_csv('../data/convfinqa_full.csv', index=False)

## Parse financial documents 

In [8]:
from docling.document_converter import DocumentConverter, PdfFormatOption
document = '../data/iag_financial report 2024.pdf'

converter = DocumentConverter()
pages = converter.convert(document).document.export_to_markdown()

2025-10-11 10:05:16,036 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-10-11 10:05:16,092 - INFO - Going to convert document batch...
2025-10-11 10:05:16,094 - INFO - Initializing pipeline for StandardPdfPipeline with options hash e647edf348883bed75367b22fbe60347
2025-10-11 10:05:16,109 - INFO - Loading plugin 'docling_defaults'
2025-10-11 10:05:16,112 - INFO - Registered picture descriptions: ['vlm', 'api']
2025-10-11 10:05:16,119 - INFO - Loading plugin 'docling_defaults'
2025-10-11 10:05:16,129 - INFO - Registered ocr engines: ['easyocr', 'ocrmac', 'rapidocr', 'tesserocr', 'tesseract']
2025-10-11 10:05:27,308 - INFO - Accelerator device: 'mps'
2025-10-11 10:05:27,318 - WARNING - Downloading detection model, please wait. This may take several minutes depending upon your network connection.
2025-10-11 10:05:41,128 - INFO - Download complete
2025-10-11 10:05:41,129 - WARNING - Downloading recognition model, please wait. This may take several minutes depending upon your netw

In [14]:
pages[0:10000]

"## ANNUAL REPORT 2024\n\n<!-- image -->\n\n## Insurance Australia Group Limited\n\nThis release has been authorised by the Board of Insurance Australia Group Limited 21 August 2024  ·  ABN 60 090 739 923\n\n<!-- image -->\n\n## Acknowledgement of Indigenous Peoples Mihi ki ngā Iwi Taketake o te Ao\n\nWe acknowledge the Traditional Owners and Elders of the different Countries across Australia and Torres Strait Islands and how Country plays a significant role in\xa0the continuation of culture and connection to land, water, and sky.\n\nIn Aotearoa New Zealand, this means we also acknowledge Tangata Whenua, their customary authority to whenua (land), and\xa0recognise the\xa0importance of te reo Māori (the Māori language), tikanga Māori (Māori customs and protocols), and te ao Māori (Māori\xa0culture\xa0and\xa0the\xa0Māori\xa0worldview).\n\n<!-- image -->\n\n## About this report\n\nThe 2024 annual report of Insurance Australia Group Limited includes IAG's full statutory accounts, along wit

In [24]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import MarkdownHeaderTextSplitter
from langchain.schema import Document
from collections import defaultdict

def merge_metadata_into_content(docs, mode="prepend"):
    """
    Adds metadata info into each Document's page_content.
    
    Args:
        docs (list[Document]): List of LangChain Document objects.
        mode (str): "prepend" or "append" — whether to add metadata before or after the content.
    """
    new_docs = []

    for doc in docs:
        # Turn metadata dict into a readable string
        meta_text = " | ".join(f"{k}: {v}" for k, v in doc.metadata.items())

        if mode == "prepend":
            new_content = f"[{meta_text}]\n\n{doc.page_content}"
        else:
            new_content = f"{doc.page_content}\n\n[{meta_text}]"

        # Create a new Document with the updated text but keep metadata
        new_docs.append(Document(page_content=new_content, metadata=doc.metadata))
    
    return new_docs

def merge_metadata(meta1, meta2):
    """
    Merge two metadata dictionaries by appending values for duplicate keys.
    Returns a new dict.
    """
    merged = defaultdict(list)

    # Add first dict
    for k, v in meta1.items():
        merged[k].append(v)

    # Add second dict
    for k, v in meta2.items():
        merged[k].append(v)

    # Flatten lists if they only contain one element
    merged = {k: v[0] if len(v) == 1 else v for k, v in merged.items()}
    return merged

def merge_small_chunks(docs, min_length=300):
    """
    Merge consecutive small chunks in a list of LangChain Documents 
    until all chunks meet the minimum content length.
    
    Args:
        docs (list[Document]): List of Document objects (already chunked).
        min_length (int): Minimum acceptable length (in characters) for each chunk.
    
    Returns:
        list[Document]: List of merged Documents.
    """
    merged_docs = []
    buffer = None  # holds ongoing merge

    for doc in docs:
        # If there's nothing in buffer, start with this doc
        if buffer is None:
            buffer = doc
            continue

        # If buffer is too small, merge it with current doc
        if len(buffer.page_content) < min_length:
            combined_text = buffer.page_content + "\n\n" + doc.page_content
            combined_meta = merge_metadata(buffer.metadata, doc.metadata)
            buffer = Document(page_content=combined_text, metadata=combined_meta)
        else:
            merged_docs.append(buffer)
            buffer = doc

    # Add any leftover buffer
    if buffer:
        merged_docs.append(buffer)
    
    return merged_docs


# Step 1: Define your markdown header structure
# You can include as many heading levels as needed
headers_to_split_on = [
    ("#", "Section"),
    ("##", "Subsection"),
    ("###", "Subsubsection")
]

# Step 2: Create the splitter
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

# Step 3: Split the markdown text into structured chunks
docs: list[Document] = markdown_splitter.split_text(pages)

# Step 4: Assuming `docs` is from MarkdownHeaderTextSplitter
docs_with_meta = merge_metadata_into_content(docs)

merged_docs_with_meta = merge_small_chunks(docs_with_meta, min_length=500)

# Step 4: Inspect the results
for i, doc in enumerate(merged_docs_with_meta[0:10]):
    print(f"\n---- Chunk {i+1} ----")
    print("Metadata:", doc.metadata)
    print("Content:\n", doc.page_content)


---- Chunk 1 ----
Metadata: {'Subsection': [['ANNUAL REPORT 2024', 'Insurance Australia Group Limited'], 'Acknowledgement of Indigenous Peoples Mihi ki ngā Iwi Taketake o te Ao']}
Content:
 [Subsection: ANNUAL REPORT 2024]

<!-- image -->

[Subsection: Insurance Australia Group Limited]

This release has been authorised by the Board of Insurance Australia Group Limited 21 August 2024  ·  ABN 60 090 739 923  
<!-- image -->

[Subsection: Acknowledgement of Indigenous Peoples Mihi ki ngā Iwi Taketake o te Ao]

We acknowledge the Traditional Owners and Elders of the different Countries across Australia and Torres Strait Islands and how Country plays a significant role inthe continuation of culture and connection to land, water, and sky.  
In Aotearoa New Zealand, this means we also acknowledge Tangata Whenua, their customary authority to whenua (land), andrecognise theimportance of te reo Māori (the Māori language), tikanga Māori (Māori customs and protocols), and te ao Māori (Māoricultu

In [26]:
type(merged_docs_with_meta[0])

langchain_core.documents.base.Document

In [29]:
# convert the parsed document to a dataframe and create an embedding for the contents
df_docs = pd.DataFrame([
    {
        "content": doc.page_content,
        "metadata": doc.metadata
    }
    for doc in merged_docs_with_meta
])

df_docs['content_embed'] = df_docs['content'].apply(lambda x: embedding_model.embed_query(x))

df_docs.head()

,content,metadata,content_embed
0,[Subsection: ANNUAL REPORT 2024]\n\n<!-- image...,"{'Subsection': [['ANNUAL REPORT 2024', 'Insura...","[0.0054753730073571205, 0.03535884618759155, -..."
1,[Subsection: About this report]\n\nThe 2024 an...,{'Subsection': 'About this report'},"[0.07063169777393341, 0.008414065465331078, -0..."
2,[Subsection: Contents]\n\n| FY24Summary ...,{'Subsection': 'Contents'},"[-0.0008982138824649155, 0.01808914914727211, ..."
3,[Subsection: 2024 Annual General Meeting]\n\nT...,"{'Subsection': ['2024 Annual General Meeting',...","[0.04234778508543968, 0.006173213943839073, -0..."
4,[Subsection: Customers]\n\n6.2M+ \nDirect cus...,"{'Subsection': [['Customers', 'Communities'], ...","[0.032148078083992004, -0.030166668817400932, ..."


In [42]:
# Putting it all together
from docling.document_converter import DocumentConverter
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import MarkdownHeaderTextSplitter
from langchain.schema import Document
from collections import defaultdict

def fin_doc_parser(document, min_chunk_length=500, docs=None):
    
    if docs is None:
        converter = DocumentConverter()
        pages = converter.convert(document).document.export_to_markdown()

        # Step 1: Define your markdown header structure
        # You can include as many heading levels as needed
        headers_to_split_on = [
            ("#", "Section"),
            ("##", "Subsection"),
            ("###", "Subsubsection")
        ]

        # Step 2: Create the splitter
        markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

        # Step 3: Split the markdown text into structured chunks
        docs: list[Document] = markdown_splitter.split_text(pages)

        # Step 4: Assuming `docs` is from MarkdownHeaderTextSplitter
        docs_with_meta = merge_metadata_into_content(docs)
    else:
        docs_with_meta = docs

    merged_docs_with_meta = merge_small_chunks(docs_with_meta, min_length=min_chunk_length)

    df_docs = pd.DataFrame([
    {
            "content": doc.page_content,
            "metadata": doc.metadata
        }
        for doc in merged_docs_with_meta
    ])

    df_docs['content_embed'] = df_docs['content'].apply(lambda x: embedding_model.embed_query(x))

    return df_docs, docs_with_meta
    

In [40]:
document = '../data/suncorp_financial report 2024.pdf'

df_docs, docs = fin_doc_parser(document, min_chunk_length=500)
df_docs.head()

2025-10-11 12:19:06,133 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-10-11 12:19:06,191 - INFO - Going to convert document batch...
2025-10-11 12:19:06,193 - INFO - Initializing pipeline for StandardPdfPipeline with options hash e647edf348883bed75367b22fbe60347
2025-10-11 12:19:06,193 - INFO - Accelerator device: 'mps'
2025-10-11 12:19:10,169 - INFO - Accelerator device: 'mps'
2025-10-11 12:19:11,762 - INFO - Accelerator device: 'mps'
2025-10-11 12:19:12,415 - INFO - Processing document suncorp_financial report 2024.pdf
2025-10-11 12:27:47,230 - INFO - Finished converting document suncorp_financial report 2024.pdf in 521.10 sec.


,content,metadata,content_embed
0,[]\n\n<!-- image -->\n\n[Subsection: Building ...,{'Subsection': ['Building futures and protecti...,"[0.034910932183265686, -0.025240078568458557, ..."
1,[Subsection: Our reporting suite]\n\nFY24 Inve...,"{'Subsection': ['Our reporting suite', 'Tax Tr...","[-0.00304177263751626, -0.01039961539208889, -..."
2,[Subsection: Performance highlights 1]\n\nGrou...,"{'Subsection': [[[['Performance highlights 1',...","[0.001944231684319675, -0.0214316938072443, -0..."
3,[Subsection: 75%]\n\nDigital sales up from 67%...,"{'Subsection': [[['75%', '$10.4m'], '99.95%'],...","[0.025364594534039497, -0.017340196296572685, ..."
4,"[Subsection: $1.42bn]\n\nEmployee salaries, su...","{'Subsection': [[['$1.42bn', '$9.7bn'], '$360m...","[-0.015178071334958076, -0.023427411913871765,..."


In [43]:
df_docs, docs = fin_doc_parser(document, min_chunk_length=800, docs=docs)

print(df_docs.iloc[3]['content'])

[Subsection: 75%]

Digital sales up from 67% 9

[Subsection: $10.4m]

Total community investment 12  
8.4/10  
Employee engagement score 6,7

[Subsection: 99.95%]

Internal Dispute Resolution (IDR) complaints resolved in 30 days 10,11

[Subsection: 76%]

Reduction in Scope 1 &amp; 2 GHG emissions from a FY20 baseline 6,13  
$1.08  
Cash earnings per share 15  
$774m Dividends paid  
9. For mass brands, Home and Motor products in Australia.
10. 30 calendar days is the ASIC maximum timeframe for Internal Dispute Resolution (IDR) responses for standard complaints, but different complaint types are subject to different maximum IDR timeframes (see RG 271.58).
11. Based on Insurance (Australia) customers. Excludes personal injury.
12. Community Investment covers cash, time, and in-kind contributions made to community causes. Verified by Business for Societal Impact (B4SI) - see verification certificate on 'B4SI' tab of Sustainability Data Pack.
13. Scope 1 and 2 emissions performance is meas

## Testing query functionality

In [2]:
df_finq = pd.read_csv('../data/convfinqa_full.csv')

In [51]:
query = "How much was the net profit of the consumer insurance segment in 2024?"
relevant_docs = find_relevant_document_fast(query, df_docs, 'content_embed', top_k=2)
for i, row in relevant_docs.iterrows():
    print(f"Document {i}:")
    print("Metadata:", row['metadata'])
    print("Content:", row['content'])
    print("\n---\n")  

Document 148:
Metadata: {'Subsection': 'Consolidated statement of comprehensive income'}
Content: [Subsection: Consolidated statement of comprehensive income]

For the financial year ended 30 June 2024  
|                                                                                                                  | Note   | 2024 $M     | Restated 2023 $M   |
|------------------------------------------------------------------------------------------------------------------|--------|-------------|--------------------|
| Insurance revenue                                                                                                | 6      | 13,697      | 12,081             |
| Insurance service expense                                                                                        | 8.1.1  | (11,321)    | (11,343)           |
| Reinsurance premium expense                                                                                      | 8.1.2  | (1,514)     | (1,422)     

In [3]:

from parsing_helpers import find_relevant_document_fast

query = "How much was the net profit of the consumer insurance segment in 2024?"
df_finq_filtered = df_finq[df_finq['has_type2_question'] == True]
relevant_finq_docs = find_relevant_document_fast(embedding_model, query, df_finq_filtered, 'summary_emb', top_k=2)
for i, row in relevant_finq_docs.iterrows():
    print(f"Document {i}:")
    print("Summary:", row['summary'])
    print("Dialogue:", row['conv_text_full'])
    print("Features:", row['features'])
    print("\n---\n")    

Document 3215:
Summary: Topic: 
Quarterly Financial Report for Consumer Packaging Division

Description: This report provides an analysis of the consumer packaging division's financial performance for 2004, 2005, and 2006, including sales, operating profits, and key drivers such as raw material costs and manufacturing efficiency.
Dialogue: """
Q: in the year of 2006, what amount from the consumer packaging sales was due to foodservice net sales?
Reason: 396
A: 396.0
Q: and what was the total of those consumer packaging sales?
Reason: 2455
A: 2455.0
Q: what percentage, then, of this total did that amount represent?
Reason: divide(396, 2455)
A: 0.1613
Q: and what was this percentage representation in the previous year, in 2005?
Reason: divide(437, 2245)
A: 0.19465
"""
Features: {'num_dialogue_turns': 4, 'has_type2_question': True, 'has_duplicate_columns': False, 'has_non_numeric_values': False}

---

Document 2715:
Summary: Topic: 
Retail Segment Financial Performance of a Company

Descr

/Users/rezamohajerpoor/Documents/Github/tomoro_use_case/rmohajer/notebooks/../src/parsing_helpers.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[embed_col] = df[embed_col].apply(parse_embedding)
